This script uses the Anthropic API to generate example sentences for vocabulary words in an Anki deck, enhancing GRE study by adding example sentences to vocab flashcards.

### Overview:
1. **Setup**: Imports necessary libraries and initializes the Claude API client with the user’s API key, AnkiConnect URL, and Anki deck name.
2. **AnkiConnect Functions**:
   - `get_notes()`: Fetches note IDs in the specified deck.
   - `get_note_info()`: Retrieves details of each note.
   - `update_note_fields()`: Updates a note’s field with a generated sentence.
3. **Formatting**: `clean_and_bold_sentence()` formats the sentence, bolding the vocabulary word.
4. **Main Process**:
   - For each note with an empty "BackAudio" field, the script sends a request to Claude Sonnet to generate an example sentence for the word in "FrontText".
   - The formatted sentence is then added to the note’s "BackAudio" field.

This process automates the creation of example sentences, adding contextual learning to Anki GRE vocab cards.

In [3]:
import requests
import json
import re
import anthropic

# Set your Claude API key here
claude_api_key = os.getenv("CLAUDE_API_KEY")
# claude_api_key = "insert_key_here"  # Replace with API key.
# TODO: It's not proper to include an API key in the script directly.
# TODO: Move the key to a virtual environment variable and retrieve it from there.

# AnkiConnect endpoint
ANKI_CONNECT_URL = 'insert_ankiconnect_url_here' # Add default AnkiConnect URL (see documentation - https://github.com/amikey/anki-connect)

# Replace with the name of your deck
deck_name = "GRE Vocab"

# Initialize Claude client
client = anthropic.Anthropic(api_key=claude_api_key)

def get_notes(deck_name):
    payload = {
        "action": "findNotes",
        "version": 6,
        "params": {
            "query": f'deck:"{deck_name}"'
        }
    }
    response = requests.post(ANKI_CONNECT_URL, json=payload).json()
    if response.get('error'):
        print(f"AnkiConnect Error: {response['error']}")
        return []
    return response['result']

def get_note_info(note_ids):
    payload = {
        "action": "notesInfo",
        "version": 6,
        "params": {
            "notes": note_ids
        }
    }
    response = requests.post(ANKI_CONNECT_URL, json=payload).json()
    if response.get('error'):
        print(f"AnkiConnect Error: {response['error']}")
        return []
    return response['result']

def update_note_fields(note_id, fields):
    payload = {
        "action": "updateNoteFields",
        "version": 6,
        "params": {
            "note": {
                "id": note_id,
                "fields": fields
            }
        }
    }
    response = requests.post(ANKI_CONNECT_URL, json=payload).json()
    if response.get('error'):
        print(f"AnkiConnect Error: {response['error']}")
    return response

def clean_and_bold_sentence(sentence, word):
    # Remove any "Here's an example..." or similar prefix
    cleaned_sentence = re.sub(r'^.*?:\s*', '', sentence)
    cleaned_sentence = re.sub(r'^Here\'s\s+', '', cleaned_sentence)
    cleaned_sentence = re.sub(r'^Let me create\s+', '', cleaned_sentence)
    
    # Remove quotes if they wrap the entire sentence
    cleaned_sentence = re.sub(r'^\s*["\'](.+)["\']$', r'\1', cleaned_sentence.strip())
    
    # Ensure the word is bolded
    escaped_word = re.escape(word)
    pattern = r"\b" + escaped_word + r"\b"
    
    # Remove any existing bold tags around the word
    cleaned_sentence = re.sub(r'</?b>', '', cleaned_sentence)
    
    # Add new bold tags
    bolded_sentence = re.sub(pattern, f"<b>{word}</b>", cleaned_sentence, flags=re.IGNORECASE)
    
    return bolded_sentence.strip()

# Main script
note_ids = get_notes(deck_name)
if not note_ids:
    print("No notes found in the deck.")
    exit()

notes_info = get_note_info(note_ids)

for note in notes_info:
    fields = note['fields']
    front_text = fields['FrontText']['value']
    back_audio = fields['BackAudio']['value']

    # Check if BackAudio is empty
    if not back_audio.strip():
        word = front_text.strip()
        
        # Create the properly formatted prompt for Claude
        messages = [
            {
                "role": "user",
                "content": f"Create a simple example sentence using the word '{word}'. Provide only the sentence, without any introduction or quotes."
            }
        ]
        
        try:
            response = client.messages.create(
                model="claude-3-sonnet-20240229",
                max_tokens=100,
                messages=messages
            )
            
            example_sentence = response.content[0].text.strip()
            
            # Clean and bold the sentence
            final_sentence = clean_and_bold_sentence(example_sentence, word)

            # Update the BackAudio field in the note
            update_note_fields(note['noteId'], {"BackAudio": final_sentence})
            print(f"Updated note with FrontText '{front_text}' with new example sentence.")
            
        except Exception as e:
            print(f"Error processing word '{word}': {str(e)}")

Updated note with FrontText 'amorphous' with new example sentence.
Updated note with FrontText 'cerebral' with new example sentence.
Updated note with FrontText 'advocate' with new example sentence.
Updated note with FrontText 'aggrandize' with new example sentence.
Updated note with FrontText 'ambivalent' with new example sentence.
Updated note with FrontText 'ameliorate' with new example sentence.
Updated note with FrontText 'amenable' with new example sentence.
Updated note with FrontText 'anachronistic' with new example sentence.
Updated note with FrontText 'audacious' with new example sentence.
Updated note with FrontText 'benign' with new example sentence.
Updated note with FrontText 'convoluted' with new example sentence.
Updated note with FrontText 'covet' with new example sentence.
Updated note with FrontText 'abate' with new example sentence.
Updated note with FrontText 'anomalous' with new example sentence.
Updated note with FrontText 'arduous' with new example sentence.
Upd